In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
!apt-get remove openjdk*

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'openjdk-11-jdk' for glob 'openjdk*'
Note, selecting 'openjdk-11-jre' for glob 'openjdk*'
Note, selecting 'openjdk-11-jre-headless' for glob 'openjdk*'
Note, selecting 'openjdk-19-jre-headless' for glob 'openjdk*'
Note, selecting 'openjdk-8-jre-zero' for glob 'openjdk*'
Note, selecting 'openjdk-11-jdk-headless' for glob 'openjdk*'
Note, selecting 'openjdk-21-jdk-headless' for glob 'openjdk*'
Note, selecting 'openjdk-19-jdk-headless' for glob 'openjdk*'
Note, selecting 'openjdk-21-demo' for glob 'openjdk*'
Note, selecting 'openjdk-18-jdk-headless' for glob 'openjdk*'
Note, selecting 'openjdk-17-dbg' for glob 'openjdk*'
Note, selecting 'openjdk-17-doc' for glob 'openjdk*'
Note, selecting 'openjdk-18-dbg' for glob 'openjdk*'
Note, selecting 'openjdk-17-jdk' for glob 'openjdk*'
Note, selecting 'openjdk-18-doc' for glob 'openjdk*'
Note, selecting 'openjdk-17-jre' for glob 'openjd

In [3]:
!apt-get update
!apt-get install openjdk-21-jdk -y

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]              
Get:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]        
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease                         
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]                 
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Package

In [4]:
!java --version

openjdk 21.0.6 2025-01-21
OpenJDK Runtime Environment (build 21.0.6+7-Ubuntu-122.04.1)
OpenJDK 64-Bit Server VM (build 21.0.6+7-Ubuntu-122.04.1, mixed mode, sharing)


In [5]:
import pandas as pd

# Load dữ liệu
vle_df = pd.read_csv('/kaggle/input/student-demographics-online-education-dataoulad/studentVle.csv')
info_df = pd.read_csv('/kaggle/input/student-demographics-online-education-dataoulad/studentInfo.csv')
vle_meta_df = pd.read_csv('/kaggle/input/student-demographics-online-education-dataoulad/vle.csv')

# Bước 1: Gộp VLE với thông tin PASS/FAIL
vle_data = pd.merge(vle_df, info_df[['id_student', 'code_module', 'code_presentation', 'final_result']],
                    on=['id_student', 'code_module', 'code_presentation'])

# Bước 2: Gộp với bảng vle để lấy loại hành động (activity_type)
vle_data = pd.merge(vle_data, vle_meta_df[['id_site', 'activity_type']], on='id_site')

In [6]:
def build_sequences_vle(group):
    sequences = []
    for student_id, grp in group.groupby('id_student'):
        seq = []
        for date, date_grp in grp.groupby('date'):
            activities = list(date_grp['activity_type'])
            seq.append(activities)
        seq = seq[-20:]
        sequences.append(seq)
    return sequences

pass_vle_seq = build_sequences_vle(vle_data[vle_data['final_result'] == 'Pass'])
fail_vle_seq = build_sequences_vle(vle_data[vle_data['final_result'] == 'Fail'])
distinction_vle_seq = build_sequences_vle(vle_data[vle_data['final_result'] == 'Distinction'])
withdrawn_vle_seq = build_sequences_vle(vle_data[vle_data['final_result'] == 'Withdrawn'])

In [7]:
pass_vle_seq_6500 = pass_vle_seq[:6500]

In [8]:
# Giả sử pass_vle_seq = [['homepage', 'forumng', 'resource'], ['homepage', 'oucontent']]
# Encode từng activity thành số ID
def encode_sequences(sequences):
    item_to_id = {}
    current_id = 1
    encoded_sequences = []
    
    for seq in sequences:  # seq là 1 sequence gồm nhiều ngày
        encoded_seq = []
        for day in seq:  # day là 1 list các activity trong ngày đó
            encoded_day = []
            homepage_seen = False
            subpage_seen = False
            
            for item in day:
                # Nếu là homepage
                if item == "homepage":
                    if homepage_seen:
                        continue  # bỏ nếu đã gặp homepage trước đó trong ngày
                    homepage_seen = True
                
                # Nếu là subpage
                elif item == "subpage":
                    if subpage_seen:
                        continue  # bỏ nếu đã gặp subpage trước đó trong ngày
                    subpage_seen = True

                # Encode như bình thường
                if item not in item_to_id:
                    item_to_id[item] = current_id
                    current_id += 1
                
                # Cho phép lặp lại các activity khác trong ngày
                encoded_day.append(item_to_id[item])
                    
            # Giữ nguyên ngày sau khi xử lý
            if encoded_day:
                encoded_seq.append(encoded_day)
        encoded_sequences.append(encoded_seq)
    
    return encoded_sequences, item_to_id


def export_spmf_format(encoded_sequences, output_file):
    with open(output_file, 'w') as f:
        for seq in encoded_sequences:
            for transaction in seq:
                line = ' '.join(map(str, transaction)) + ' -1 '
                f.write(line)
            f.write('-2\n')

encoded_pass_seqs, item_map = encode_sequences(pass_vle_seq_6500)
output_file = "/kaggle/working/pass_vle_spmf_6500.txt"
export_spmf_format(encoded_pass_seqs, output_file)

In [9]:
def encode_sequences_item_map(sequences, item_map):
    encoded_sequences = []

    for seq in sequences:  # seq là 1 sequence gồm nhiều ngày
        encoded_seq = []
        for day in seq:  # day là 1 list các activity trong ngày đó
            encoded_day = []
            homepage_seen = False
            subpage_seen = False

            for item in day:
                item_id = item_map.get(item, None)  # None nếu item không tồn tại trong item_map
                if item_id is None:
                    continue  # Nếu không map được thì bỏ qua luôn

                # Giới hạn homepage và subpage chỉ 1 lần trong ngày
                if item == "homepage":
                    if homepage_seen:
                        continue
                    homepage_seen = True
                elif item == "subpage":
                    if subpage_seen:
                        continue
                    subpage_seen = True
                encoded_day.append(item_id)
                
            if encoded_day:
                encoded_seq.append(encoded_day)
        encoded_sequences.append(encoded_seq)

    return encoded_sequences


encoded_fail_seqs = encode_sequences_item_map(fail_vle_seq,item_map)
output_file = "/kaggle/working/fail_vle_spmf.txt"
export_spmf_format(encoded_fail_seqs, output_file)

encoded_distinction_seqs = encode_sequences_item_map(distinction_vle_seq,item_map)
output_file = "/kaggle/working/distinction_vle_spmf.txt"
export_spmf_format(encoded_distinction_seqs, output_file)

encoded_withdrawn_seqs = encode_sequences_item_map(withdrawn_vle_seq,item_map)
output_file = "/kaggle/working/withdrawn_vle_spmf.txt"
export_spmf_format(encoded_withdrawn_seqs, output_file)

In [10]:
print(item_map)

{'url': 1, 'oucontent': 2, 'homepage': 3, 'subpage': 4, 'forumng': 5, 'resource': 6, 'quiz': 7, 'page': 8, 'oucollaborate': 9, 'externalquiz': 10, 'ouwiki': 11, 'folder': 12, 'glossary': 13, 'questionnaire': 14, 'dualpane': 15, 'dataplus': 16, 'ouelluminate': 17, 'htmlactivity': 18, 'sharedsubpage': 19}


In [11]:
import subprocess
import re
import matplotlib.pyplot as plt
algorithms = ["PrefixSpan", "GSP", "SPADE"]
input_files = ["/kaggle/working/pass_vle_spmf_6500.txt","/kaggle/working/fail_vle_spmf.txt","/kaggle/working/distinction_vle_spmf.txt","/kaggle/working/withdrawn_vle_spmf.txt"]
output_files = []
results_table = []
for input_file in input_files:
    for algo in algorithms:
        output_file = f"output_{input_file.split('/')[3].split('_')[0]}_{algo}.txt"
        output_files.append(output_file)
        command = ["java", "-jar", "/kaggle/input/spmfjava/spmf.jar", "run", algo, input_file, output_file, "50%"]
        print(f"Running command: {' '.join(command)}")  # In lệnh
        result = subprocess.run(command, capture_output=True, text=True)

        total_time_match = re.search(r"Total time ~ (\d+) ms", result.stdout)
        sequence_count_match = re.search(r"Frequent sequences count : (\d+)", result.stdout)
        
        
        if total_time_match and sequence_count_match:
            # Convert milliseconds to seconds
            total_time_seconds = int(total_time_match.group(1)) / 1000
            results_table.append({
                "Algorithm": algo,
                "Support (%)": '50%',
                "Total Time (s)": total_time_seconds,
                "Frequent Sequences Count": int(sequence_count_match.group(1))
            })

print(f"{'Algorithm':<12}{'Support (%)':<12}{'Total Time (s)':<15}{'Sequences Count':<18}")
print("=" * 60)
for result in results_table:
    print(f"{result['Algorithm']:<12}{result['Support (%)']:<12}{result['Total Time (s)']:<15.3f}{result['Frequent Sequences Count']:<18}")


Running command: java -jar /kaggle/input/spmfjava/spmf.jar run PrefixSpan /kaggle/working/pass_vle_spmf_6500.txt output_pass_PrefixSpan.txt 50%
Running command: java -jar /kaggle/input/spmfjava/spmf.jar run GSP /kaggle/working/pass_vle_spmf_6500.txt output_pass_GSP.txt 50%
Running command: java -jar /kaggle/input/spmfjava/spmf.jar run SPADE /kaggle/working/pass_vle_spmf_6500.txt output_pass_SPADE.txt 50%
Running command: java -jar /kaggle/input/spmfjava/spmf.jar run PrefixSpan /kaggle/working/fail_vle_spmf.txt output_fail_PrefixSpan.txt 50%
Running command: java -jar /kaggle/input/spmfjava/spmf.jar run GSP /kaggle/working/fail_vle_spmf.txt output_fail_GSP.txt 50%
Running command: java -jar /kaggle/input/spmfjava/spmf.jar run SPADE /kaggle/working/fail_vle_spmf.txt output_fail_SPADE.txt 50%
Running command: java -jar /kaggle/input/spmfjava/spmf.jar run PrefixSpan /kaggle/working/distinction_vle_spmf.txt output_distinction_PrefixSpan.txt 50%
Running command: java -jar /kaggle/input/spmfj

In [12]:
def decode_pattern(line, item_map):
    reverse_map = {v: k for k, v in item_map.items()}
    tokens = line.strip().split(' ')
    decoded = []
    for token in tokens:
        if token.isdigit():
            decoded.append(reverse_map.get(int(token), token))
        else:
            decoded.append(token)
    return ' '.join(decoded)

for output_file in output_files:
    with open(output_file, 'r') as infile:
        lines = infile.readlines()
    
    decoded_lines = [decode_pattern(line, item_map) for line in lines]
    
    # Tạo file output mới để lưu kết quả giải mã
    decoded_output_file = f"decoded_{output_file}_50%"
    with open(decoded_output_file, 'w') as outfile:
        outfile.write('\n'.join(decoded_lines))
    
    print(f"Decoded file saved to: {decoded_output_file}")

Decoded file saved to: decoded_output_pass_PrefixSpan.txt_50%
Decoded file saved to: decoded_output_pass_GSP.txt_50%
Decoded file saved to: decoded_output_pass_SPADE.txt_50%
Decoded file saved to: decoded_output_fail_PrefixSpan.txt_50%
Decoded file saved to: decoded_output_fail_GSP.txt_50%
Decoded file saved to: decoded_output_fail_SPADE.txt_50%
Decoded file saved to: decoded_output_distinction_PrefixSpan.txt_50%
Decoded file saved to: decoded_output_distinction_GSP.txt_50%
Decoded file saved to: decoded_output_distinction_SPADE.txt_50%
Decoded file saved to: decoded_output_withdrawn_PrefixSpan.txt_50%
Decoded file saved to: decoded_output_withdrawn_GSP.txt_50%
Decoded file saved to: decoded_output_withdrawn_SPADE.txt_50%
